A companion to:
[Transit Data Quality Analysis Standard Operating Procedures ](https://caltrans.sharepoint.com/:w:/s/DOTPMPHQ-DataandDigitalServices/Ec2SNnePpLlHsSayRT-ptc0B0JbEtVxRKrz-3lWG26v_JQ?e=cx8QUr)


In [1]:
from dotenv import load_dotenv
import json
import os

import pandas as pd
import requests

from calitp_data_analysis.sql import to_snakecase
import script

In [2]:
load_dotenv()
MTC_API_KEY = os.getenv('MTC_API_KEY')
mtc_operator_url = f"https://api.511.org/transit/operators?api_key={MTC_API_KEY}"

In [3]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

## Airtable

In [4]:
airtable = script.load_airtable(
    "./Transit Data Quality Issues-Open Issues by Distrcit.csv"
)

## Catastrophic

In [5]:
cat_df = script.load_catastrophic_errors(
    "./feed_info_expired_feeds_2024-02-08T16_25_33.855497Z.xlsx"
)

/opt/conda/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [6]:
cat_df

,service_name,district,uri,date
13,Victor Valley Transit,8,https://ontime.vvta.org/gtfs,2024-02-05
19,Victor Valley Transit,8,https://ontime.vvta.org/gtfs,2024-02-04
25,Victor Valley Transit,8,https://ontime.vvta.org/gtfs,2024-02-03
31,Victor Valley Transit,8,https://ontime.vvta.org/gtfs,2024-02-02
1,Tahoe Transportation District,3,https://data.trilliumtransit.com/gtfs/tahoe-ca-us/tahoe-ca-us.zip,2024-02-07
2,Tahoe Transportation District,3,https://data.trilliumtransit.com/gtfs/tahoe-ca-us/tahoe-ca-us.zip,2024-02-07
3,Tahoe Transportation District,3,https://data.trilliumtransit.com/gtfs/tahoe-ca-us/tahoe-ca-us.zip,2024-02-07
5,Tahoe Transportation District,3,https://data.trilliumtransit.com/gtfs/tahoe-ca-us/tahoe-ca-us.zip,2024-02-06
6,Tahoe Transportation District,3,https://data.trilliumtransit.com/gtfs/tahoe-ca-us/tahoe-ca-us.zip,2024-02-06
7,Tahoe Transportation District,3,https://data.trilliumtransit.com/gtfs/tahoe-ca-us/tahoe-ca-us.zip,2024-02-06


In [7]:
script.summarize_cat(cat_df, airtable)

,service_name,uri,# of days with expired feed
0,Fresno County Rural Transit Agency,https://rapid.nationalrtap.org/GTFSFileManagement/UserUploadFiles/7869/google_transit.zip,12
1,San Joaquin Regional Transit District,http://sanjoaquinrtd.com/RTD-GTFS/RTD-GTFS.zip,4
2,Tahoe Transportation District,https://data.trilliumtransit.com/gtfs/tahoe-ca-us/tahoe-ca-us.zip,42
3,Victor Valley Transit,https://ontime.vvta.org/gtfs,4


,service_name,uri,# of days with expired feed,gtfs_datasets,services,issue_type,description,airtable_ticket
0,Fresno County Rural Transit Agency,https://rapid.nationalrtap.org/GTFSFileManagement/UserUploadFiles/7869/google_transit.zip,12,Fresno County Schedule,Fresno County Rural Transit Agency,About to Expire Schedule Feed,The GTFS schedule feed is about to expire on 01/01/2024,Yes
1,San Joaquin Regional Transit District,http://sanjoaquinrtd.com/RTD-GTFS/RTD-GTFS.zip,4,NaN,NaN,NaN,NaN,NaN
2,Tahoe Transportation District,https://data.trilliumtransit.com/gtfs/tahoe-ca-us/tahoe-ca-us.zip,42,NaN,NaN,NaN,NaN,NaN
3,Victor Valley Transit,https://ontime.vvta.org/gtfs,4,NaN,NaN,NaN,NaN,NaN


## Incompleteness

In [8]:
r = requests.get(mtc_operator_url)
df_511 = pd.DataFrame(json.loads(r.content))

Bay Area feeds that Real Time feeds are monitored

In [9]:
df_511 = pd.DataFrame(json.loads(r.content))
df_511.loc[df_511['Monitored'].values,][['Name']]

,Name
4,AC TRANSIT
7,Bay Area Rapid Transit
8,Caltrain
12,County Connection
13,Dumbarton Express Consortium
15,FAST
17,Golden Gate Transit
18,Livermore Amador Valley Transit Authority
19,Marin Transit
21,MVgo


In [10]:
airtable.loc[airtable.issue_type.str.contains("Completeness")]

,gtfs_datasets,services,issue_type,description,airtable_ticket
8,"Bay Area 511 Emery Go-Round VehiclePositions,Bay Area 511 Emery Go-Round TripUpdates",Emery Go-Round,GTFS Realtime Completeness Problem,"As of December 15, Bay Area 511 Emery Go-Round GTFS-RT has recorded 0% in the past 14 days. Since November 30, TripUpdates and VehiclePositions has reported 0% trip data.",Yes
9,Bay Area 511 Union City Transit VehiclePositions,Union City Transit,GTFS Realtime Completeness Problem,"In the past 14 days, GTFS-RT Vehicle Positions is 37.21%, which is a reduction from 46.15% VP recorded on August 25, 2023. Also, September GTFS-RT Vehicle Positions activity has fluctuated from 31.41% to 41.67%.",Yes
10,Bay Area 511 Vine Transit VehiclePositions,Vine Transit,GTFS Realtime Completeness Problem,GTFS-Realtime Vehicle Position data production has not been consistent throughout the month. The last two weeks' average Vehicle Position data production was 39.86%,Yes
17,Desert Roadrunner GMV Schedule,Desert Roadrunner,GTFS Realtime Completeness Problem,"As of January 5, Desert Roadrunner GMV Schedule recorded 21.78% TripUpdates and 20.83% VehiclePositions in the past 14 days. From December 25 through January 4, TripUpdates and VehiclePosition fluctuated and reported less than 44% of trip data.",Yes
34,"MVGO VehiclePositions,MVGO TripUpdates",MVGO,GTFS Realtime Completeness Problem,Since July 3 there have been zero trips accounted for in both the Vehicle Positions and Trip Updates for MVGO\n\n,Yes
37,Merced VehiclePositions,Merced The Bus,GTFS Realtime Completeness Problem,Transit Joint Powers Authority for Merced County: GTFS-RT issue: complete outage of GTFS-RT Vehicle Positions Data,Yes
43,"Santa Ynez Vehicle Positions,Santa Ynez Mecatran Schedule",Santa Ynez Valley Transit,GTFS Realtime Completeness Problem,"As of 11/16: Only 39% of trips have vehicle position messages, but TU is ok.",Yes
45,"SunLine Vehicle Positions,SunLine Trip Updates",SunLine Transit,GTFS Realtime Completeness Problem,"Since May 4, Sunline's Trip Updates and Vehicle Positions have consistently accounted for only 45% of scheduled trips.\n\n",Yes


In [18]:
airtable.loc[airtable.gtfs_datasets.str.contains("Down")]

,gtfs_datasets,services,issue_type,description,airtable_ticket
18,"DowneyLINK GMV Trip Updates,DowneyLINK GMV Vehicle Positions",DowneyLINK,None,"As of 1/16/2024, 0% of trips are recording trip updates and vehicle positions.",Yes
19,DowneyLINK Schedule Historic,DowneyLINK,Los Angeles Metro feed transition,City of Downey: Los Angeles Metro feed to transition to Cal-ITP maintenance,Yes


In [11]:
script.incomplete(
    "./gtfs_rt_trip_updates_completeness__last_14_days__2024-02-08T16_25_24.133437Z.xlsx",
    "./gtfs_rt_vehicle_positions_completeness__last_14_days__2024-02-08T16_25_26.305133Z.xlsx",
    airtable,
    41,
)

/opt/conda/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,name,%_of_trips_with_tu_messages,%_of_trips_with_vp_messages,gtfs_datasets,services,issue_type,description,airtable_ticket,_merge
0,Anaheim Resort Schedule,10.31,8.91,Anaheim Resort Schedule,Anaheim Resort Transportation,Service Accuracy,Anaheim Transportation Network: GTFS-Schedule issue: thousands of duplicate trips on Toy Story Line,Yes,both
1,Bay Area 511 Angel Island-Tiburon Ferry Schedule,0.00,0.00,NA,NA,NA,NA,NA,problems_w_tu_only
2,Bay Area 511 BART Schedule,OK,0.00,NA,NA,NA,NA,NA,problems_w_tu_only
3,Bay Area 511 Commute.org Schedule,0.00,0.00,NA,NA,NA,NA,NA,problems_w_tu_only
4,Bay Area 511 Golden Gate Ferry Schedule,0.00,0.00,NA,NA,NA,NA,NA,problems_w_tu_only
5,Bay Area 511 MVGO Schedule,25.00,25.00,NA,NA,NA,NA,NA,problems_w_tu_only
6,Bay Area 511 Mission Bay Schedule,0.00,0.00,NA,NA,NA,NA,NA,problems_w_tu_only
7,Bay Area 511 Rio Vista Delta Breeze Schedule,0.00,0.00,NA,NA,NA,NA,NA,problems_w_tu_only
8,Bay Area 511 SFO AirTrain Schedule,0.00,0.00,NA,NA,NA,NA,NA,problems_w_tu_only
9,Bay Area 511 South San Francisco Shuttle Schedule,0.00,0.00,NA,NA,NA,NA,NA,problems_w_tu_only


In [16]:
# Unfiltered version of the incompleteness dataq
incompleteness_complete = script.incomplete(
    "./gtfs_rt_trip_updates_completeness__last_14_days__2024-02-08T16_25_24.133437Z.xlsx",
    "./gtfs_rt_vehicle_positions_completeness__last_14_days__2024-02-08T16_25_26.305133Z.xlsx",
    airtable,
    100,
)

/opt/conda/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [19]:
incompleteness_complete.name.unique()

array(['AC Transit Schedule', 'Anaheim Resort Schedule',
       'Antelope Valley Transit Authority Schedule', 'Arcadia Schedule',
       'BART Schedule', 'Banning Pass Schedule',
       'Bay Area 511 AC Transit Schedule', 'Bay Area 511 ACE Schedule',
       'Bay Area 511 Angel Island-Tiburon Ferry Schedule',
       'Bay Area 511 BART Schedule', 'Bay Area 511 Caltrain Schedule',
       'Bay Area 511 Capitol Corridor Schedule',
       'Bay Area 511 Commute.org Schedule',
       'Bay Area 511 County Connection Schedule',
       'Bay Area 511 Dumbarton Express Schedule',
       'Bay Area 511 Emery Go-Round Schedule',
       'Bay Area 511 Fairfield and Suisun Transit Schedule',
       'Bay Area 511 Golden Gate Ferry Schedule',
       'Bay Area 511 Golden Gate Transit Schedule',
       'Bay Area 511 MVGO Schedule', 'Bay Area 511 Marin Schedule',
       'Bay Area 511 Mission Bay Schedule', 'Bay Area 511 Muni Schedule',
       'Bay Area 511 Petaluma Schedule', 'Bay Area 511 Regional Schedule',

In [20]:
ops_to_check = [
    "Santa Ynez Mecatran Schedule",
    "SunLine Avail Schedule",
 'Merced GMV Schedule',
    'Bay Area 511 MVGO Schedule',
    'Desert Roadrunner GMV Schedule',
     'Bay Area 511 Vine Transit Schedule',
     'Bay Area 511 Union City Transit Schedule',
      'Bay Area 511 Emery Go-Round Schedule',
]

In [21]:
incompleteness_complete.loc[incompleteness_complete.name.isin(ops_to_check)]

,name,%_of_trips_with_tu_messages,%_of_trips_with_vp_messages,gtfs_datasets,services,issue_type,description,airtable_ticket,_merge
15,Bay Area 511 Emery Go-Round Schedule,73.33,72.99,NA,NA,NA,NA,NA,problems_w_tu_only
19,Bay Area 511 MVGO Schedule,25.00,25.00,NA,NA,NA,NA,NA,problems_w_tu_only
38,Bay Area 511 Union City Transit Schedule,76.48,48.36,NA,NA,NA,NA,NA,problems_w_tu_only
40,Bay Area 511 Vine Transit Schedule,54.12,49.04,NA,NA,NA,NA,NA,problems_w_tu_only
50,Desert Roadrunner GMV Schedule,75.76,71.02,Desert Roadrunner GMV Schedule,Desert Roadrunner,GTFS Realtime Completeness Problem,"As of January 5, Desert Roadrunner GMV Schedule recorded 21.78% TripUpdates and 20.83% VehiclePositions in the past 14 days. From December 25 through January 4, TripUpdates and VehiclePosition fluctuated and reported less than 44% of trip data.",Yes,both
70,Merced GMV Schedule,OK,0.00,NA,NA,NA,NA,NA,problems_w_tu_only
95,Santa Ynez Mecatran Schedule,OK,47.11,NA,NA,NA,NA,NA,problems_w_tu_only
99,SunLine Avail Schedule,50.96,45.00,NA,NA,NA,NA,NA,problems_w_tu_only
